### アノテーションデータをbrat形式に変換する

### データベース接続の関数定義

In [3]:
import json
import sqlite3

conn = None

# データベース接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('../../NaturalLanguageProcessing/data/sqlite3/sqlite3')

# データベース接続終了
def close():
#   終了
    conn.close()

# テーブル作成
def create_table():
#   executeでSQL構文作成、docsがあれば削除
    conn.execute('DROP TABLE IF EXISTS docs')
#   docsテーブルを新規作成
    conn.execute('''CREATE TABLE docs (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            content     TEXT,
            meta_info   BLOB,
            sentence    BLOB,
            chunk       BLOB,
            token       BLOB
        )''')

# データをインサートする
def db_load(values):
    
#   valuesに指定したパラメータ順序またはマッピングを?に入れて実行する
    conn.executemany(
        'INSERT INTO docs (content, meta_info) VALUES (?,?)',
        values)
#   確定
    conn.commit()

# 一部のデータを見る
def get(doc_id, fl):
#   .fetchone()でカーソルの次の行を取得
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    row_dict = {}
#   flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# id番号を抜き出す
def get_all_ids(limit, offset=0):
#   limitで取得上限、OFFSETで開始位置を指定してデータを抜き出す。そのデータの1番目id番号を抜き出す
    return [record[0] for record in conn.execute('SELECT id FROM docs LIMIT ? OFFSET ?', (limit, offset))]

#### アノテーション抜き出し関数

In [4]:
# アノテーションを確認
def get_annotation(doc_id, name):
#   docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute('SELECT {0} FROM docs WHERE id = ?'.format(name), (doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

### アノテーションをbrat用のデータの形式に変換したファイルを作成する。

In [5]:
connect()
anno_name = 'affiliation'

for doc_id in get_all_ids(limit=-1):
    
    text = get(doc_id, fl=['content'])['content']
    with open('../0Day-data/brat/{0}.txt'.format(doc_id), 'w') as f:
        f.write(text)
    with open('../0Day-data/brat/{0}.ann'.format(doc_id), 'w') as f:
        for i, anno in enumerate(get_annotation(doc_id, anno_name)):
            f.write('T{0}\t{1} {2} {3}\t{4}\n'.format(
                i,
                'affiliation',
                anno['begin'],
                anno['end'],
                text[anno['begin']:anno['end']]
            ))
close()


FileNotFoundError: [Errno 2] No such file or directory: '../0Day-data/brat/1.txt'